This code was based on in the following references:

[1] "A Novel Search Algorithm based on Fish School Behavior" published in 2008 by Bastos Filho, Lima Neto, Lins, D. O. Nascimento and P. Lima

[2] "An Enhanced Fish School Search Algorithm" published in 2013 by Bastos Filho and  D. O. Nascimento

[3] "Defining a Standard for Particle Swarm Optimization" published in 2007 by Bratton and Kennedy


### Imports

In [ ]:
import os
import sys
import csv
import random
import copy
import math
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from random import randint
from google.colab import drive, files
%matplotlib inline
plt.rcParams["figure.figsize"] = (13,8)

### Connection with Google Drive

In [ ]:
drive.mount('/content/drive', force_remount=True) #  Connect to drive
FOLDERNAME = 'POLI/decimo/tcc/Algoritms/Algoritms Notebook/' #  Set folder that have the datasts
assert FOLDERNAME is not None, "[!] Enter the foldername."
sys.path.append('/content/drive/My Drive/{}'.format(FOLDERNAME)) #  Locale the foldeer

%cd /content/drive/My\ Drive/$FOLDERNAME

Mounted at /content/drive
/content/drive/My Drive/POLI/decimo/tcc/Algoritms/Algoritms Notebook


### Class of Fish and School Fish with each method to execute optimization

In [ ]:
from ObjectiveFunction import *
from SearchSpaceInitializer import UniformSSInitializer, OneQuarterDimWiseSSInitializer
%cd ../

class Fish(object):
  """
  Classe que define um peixe que contém sua posição (em diversas dimensões),
  seu peso e custo de desempenho. Para auxilio também memoriza o quanto andou
  e quanto de custo mudou.
  """
  def __init__(self, dim):
    nan = float('nan')
    self.pos = [nan for _ in range(dim)]
    self.last_pos = [nan for _ in range(dim)]
    self.delta_pos = np.nan
    self.delta_cost = np.nan
    self.weight = np.nan
    self.cost = np.nan
    # self.has_improved = False

class FSS(object):
  """
  Classe que representa a escola de peixes (classe acima) que irá se locomover
  dentro de uma região e serão avaliados de acordo com uma função objetivo
  """
  def __init__(self, objective_function, space_initializer, n_iter, school_size,
               step_ind_init, step_ind_final, step_vol_init, step_vol_final,
               min_weight, weight_scale, analytic_in=False):
    self.objective_function = objective_function # função de avalição de custo
    self.space_initializer = space_initializer # posições iniciais dos peixes

    self.dim = objective_function.dim
    self.minf = objective_function.minf # limite minimo da função
    self.maxf = objective_function.maxf # limite máximo da função
    self.n_iter = n_iter
    self.barycenter = np.zeros(self.dim)

    self.school_size = school_size  # quantidade de peixes
    self.step_ind_init = step_ind_init
    self.step_ind_final = step_ind_final
    self.step_vol_init = step_vol_init
    self.step_vol_final = step_vol_final

    self.curr_step_ind = self.step_ind_init * (self.maxf - self.minf)
    self.curr_step_vol = self.step_vol_init * (self.maxf - self.minf)
    self.min_w = min_weight
    self.w_scale = weight_scale
    self.prev_weight_school = 0.0
    self.curr_weight_school = 0.0
    self.best_fish = None
    
    self.analytic_in = analytic_in
    self.i_net = []
    
    self.optimum_cost_tracking_iter = []
    self.optimum_cost_tracking_eval = []

  def __gen_weight(self):
    return self.w_scale / 2.0

  def __init_fss(self):
    self.optimum_cost_tracking_iter = []
    self.optimum_cost_tracking_eval = []

  def __init_fish(self, pos):
    fish = Fish(self.dim)
    fish.pos = pos
    fish.weight = self.__gen_weight()
    fish.cost = self.objective_function.evaluate(fish.pos)
    self.optimum_cost_tracking_eval.append(self.best_fish.cost)
    return fish

  def __init_school(self):
    self.best_fish = Fish(self.dim)
    self.best_fish.cost = np.inf
    self.curr_weight_school = 0.0
    self.prev_weight_school = 0.0
    self.school = []

    positions = self.space_initializer.sample(self.objective_function, self.school_size)

    for idx in range(self.school_size):
        fish = self.__init_fish(positions[idx])
        self.school.append(fish)
        self.curr_weight_school += fish.weight
    self.prev_weight_school = self.curr_weight_school
    self.update_best_fish()
    self.optimum_cost_tracking_iter.append(self.best_fish.cost)
  
  def update_best_fish(self):
    for fish in self.school:
      if self.best_fish.cost > fish.cost:
        self.best_fish = copy.copy(fish)

  def update_steps(self, curr_iter):
    self.curr_step_ind = self.step_ind_init - curr_iter * float(self.step_ind_init - self.step_ind_final) / self.n_iter
    self.curr_step_vol = self.step_vol_init - curr_iter * float(self.step_vol_init - self.step_vol_final) / self.n_iter

  def max_delta_cost(self):
    max_ = 0
    for fish in self.school:
      if max_ < fish.delta_cost:
        max_ = fish.delta_cost
    return max_
    
  def total_school_weight(self):
    self.prev_weight_school = self.curr_weight_school
    self.curr_weight_school = 0.0
    for fish in self.school:
      self.curr_weight_school += fish.weight
      
  def calculate_barycenter(self):
    barycenter = np.zeros((self.dim,), dtype=float)
    density = 0.0

    for fish in self.school:
      density += fish.weight
      for dim in range(self.dim):
        barycenter[dim] += (fish.pos[dim] * fish.weight)
    for dim in range(self.dim):
      barycenter[dim] = barycenter[dim] / density

    return barycenter

  def feeding(self):
    for fish in self.school:
      if self.max_delta_cost():
        fish.weight = fish.weight + (fish.delta_cost / self.max_delta_cost())
      if fish.weight > self.w_scale:
        fish.weight = self.w_scale
      elif fish.weight < self.min_w:
        fish.weight = self.min_w
  
  def individual_movement(self, n_iter):
    for fish in self.school:
      new_pos = np.zeros((self.dim,), dtype=float)
      for dim in range(self.dim):
        new_pos[dim] = fish.pos[dim] + (self.curr_step_ind * np.random.uniform(-1, 1))
        if new_pos[dim] < self.minf:
            new_pos[dim] = self.minf
        elif new_pos[dim] > self.maxf:
            new_pos[dim] = self.maxf
      
      if n_iter == 0:
        cost = self.objective_function.evaluate(new_pos)
        self.optimum_cost_tracking_eval.append(self.best_fish.cost)
        if cost < fish.cost:
          fish.delta_cost = abs(cost - fish.cost)
          fish.cost = cost
          delta_pos = np.zeros((self.dim,), dtype=float)
          for idx in range(self.dim):
            delta_pos[idx] = new_pos[idx] - fish.pos[idx]
          fish.last_pos = copy.copy(fish.pos)
          fish.delta_pos = delta_pos
          fish.pos = new_pos
        else:
          fish.delta_pos = np.zeros((self.dim,), dtype=float)
          fish.delta_cost = 0
      else:
        fish.pos = new_pos
          
  def collective_instinctive_movement(self):
    cost_eval_enhanced = np.zeros((self.dim,), dtype=float)
    density = 0.0
    for fish in self.school:
      density += fish.delta_cost
      for dim in range(self.dim):
        cost_eval_enhanced[dim] += (fish.delta_pos[dim] * fish.delta_cost)
    for dim in range(self.dim):
      if density != 0:
        cost_eval_enhanced[dim] = cost_eval_enhanced[dim] / density
    for fish in self.school:
      new_pos = np.zeros((self.dim,), dtype=float)
      for dim in range(self.dim):
        new_pos[dim] = fish.pos[dim] + cost_eval_enhanced[dim]
        if new_pos[dim] < self.minf:
          new_pos[dim] = self.minf
        elif new_pos[dim] > self.maxf:
          new_pos[dim] = self.maxf

        fish.pos = new_pos
  
  def collective_volitive_movement(self, n_iter):
    self.total_school_weight()
    barycenter = self.calculate_barycenter()
    self.barycenter = barycenter
    for fish in self.school:
      new_pos = np.zeros((self.dim,), dtype=float)
      for dim in range(self.dim):
        if self.curr_weight_school > self.prev_weight_school:
          new_pos[dim] = fish.pos[dim] - ((fish.pos[dim] - barycenter[dim]) * self.curr_step_vol * np.random.uniform(0, 1))
        else:
          new_pos[dim] = fish.pos[dim] + ((fish.pos[dim] - barycenter[dim]) * self.curr_step_vol * np.random.uniform(0, 1))
        if new_pos[dim] < self.minf:
          new_pos[dim] = self.minf
        elif new_pos[dim] > self.maxf:
          new_pos[dim] = self.maxf
      
      if n_iter > 0:
        cost = self.objective_function.evaluate(new_pos)
        self.optimum_cost_tracking_eval.append(self.best_fish.cost)
        if cost < fish.cost:
          fish.delta_cost = abs(cost - fish.cost)
          fish.cost = cost
          delta_pos = np.zeros((self.dim,), dtype=float)
          for idx in range(self.dim):
            delta_pos[idx] = new_pos[idx] - fish.pos[idx]
          fish.last_pos = copy.copy(fish.pos)
          fish.delta_pos = delta_pos
          fish.pos = new_pos
        else:
          fish.delta_pos = np.zeros((self.dim,), dtype=float)
          fish.delta_cost = 0
      else:
        fish.pos = new_pos


      # cost = self.objective_function.evaluate(new_pos)
      # self.optimum_cost_tracking_eval.append(self.best_fish.cost)
      # fish.cost = cost
      # fish.pos = new_pos
      
  def optimize(self):
    self.__init_fss()
    self.__init_school()

    for i in range(self.n_iter):
      self.individual_movement(i)
      self.update_best_fish()
      self.collective_instinctive_movement()
      self.collective_volitive_movement(i)
      self.feeding()
      self.update_steps(i)
      self.update_best_fish()
      self.optimum_cost_tracking_iter.append(self.best_fish.cost)
      #print "Iteration: ", i, " Cost: ", self.best_fish.cost

/content/drive/My Drive/POLI/decimo/tcc/Algoritms


### Validate FSS

Validate learning in each iteration

In [ ]:
def create_dir(path):
  directory = os.path.dirname(path)
  try:
    os.stat(directory)
  except:
    os.mkdir(directory)

def main():
  for d in [15, 30, 50, 100]:
    print (f"starting FSS ({d})")
    search_space_initializer = UniformSSInitializer()
    result_path = os.path.dirname(os.path.abspath('Algoritms')) + os.sep + "Results" + os.sep + f"{d}d" + os.sep
    num_exec = 1
    school_size = 30
    num_iterations = 1000
    step_individual_init = 0.1
    step_individual_final = 0.0001
    step_volitive_init = 0.01
    step_volitive_final = 0.001
    min_w = 1
    w_scale = num_iterations / 2.0

    unimodal_funcs = [SphereFunction, RotatedHyperEllipsoidFunction, RosenbrockFunction, DixonPriceFunction, QuarticNoiseFunction]
    multimodal_funcs = [GeneralizedShwefelFunction, RastriginFunction, AckleyFunction, GriewankFunction, LeviFunction]
    regular_functions = unimodal_funcs + multimodal_funcs

    cec_functions = []

    create_dir(result_path)
    f_handle_csv = open(result_path + "/FSS_iter.csv", 'w+')
    writer_csv = csv.writer(f_handle_csv, delimiter=",")
    header = ['opt', 'func', 'exec_time'] + [f"run{str(i+1)}" for i in range(num_iterations)]
    writer_csv.writerow(header)

    for benchmark_func in regular_functions:
      func = benchmark_func(d)
      start = time.time()
      bests_iter, bests_eval = run_experiments(num_iterations, school_size, num_exec, func,
                            search_space_initializer, step_individual_init,
                            step_individual_final, step_volitive_init,
                            step_volitive_final, min_w, w_scale)
      end = time.time()
      row_csv = ['FSS', func.function_name, (end - start)] + [(b if b != math.inf else 9.9e+999) for b in bests_iter[:num_iterations]]
      writer_csv.writerow(row_csv)
      print(func.function_name, end - start, bests_iter[-1])
      
    f_handle_csv.close()

def run_experiments(n_iter, school_size, num_runs, objective_function,
                    search_space_initializer, step_individual_init,
                    step_individual_final, step_volitive_init,
                    step_volitive_final, min_w, w_scale):
  opt1 = FSS(objective_function=objective_function, space_initializer=search_space_initializer,
              n_iter=n_iter, school_size=school_size, step_ind_init=step_individual_init,
              step_ind_final=step_individual_final, step_vol_init=step_volitive_init,
              step_vol_final=step_volitive_final, min_weight=min_w, weight_scale=w_scale)
  opt1.optimize()
  return opt1.optimum_cost_tracking_iter, opt1.optimum_cost_tracking_eval

main()

starting FSS (15)
Sphere 5.456316232681274 1.4132742138400182e-10
RotatedHyperEllipsoid 7.912359714508057 9.503107639555328e-06
Rosenbrock 6.373836040496826 13.771109691353477
Dixon-Price 5.847585201263428 0.37500000510486364
Quartic-Noise 5.444572925567627 0.0001517265727979346
Generalized-Shwefel 5.317721366882324 4125.385684137842
Rastrigin 5.534265041351318 4.7909741113016935e-05
Ackley 5.853510856628418 0.0001285074915036155
Griewank 6.824621200561523 0.008155330981703846
Levi 7.414424419403076 2.1605028095786336
starting FSS (30)
Sphere 10.129152536392212 1.33821633228569e-07
RotatedHyperEllipsoid 15.732733964920044 4.780291452811371e-06
Rosenbrock 12.096024990081787 28.77110968964552
Dixon-Price 11.60185194015503 0.37500000015954915
Quartic-Noise 10.487730979919434 0.0006320364457120905
Generalized-Shwefel 10.017302751541138 10257.332194458428
Rastrigin 10.68487024307251 1.57251458290375e-05
Ackley 10.541001319885254 1.7348571948350155e-05
Griewank 10.928996562957764 0.009692585

Validate bests results in multiple sessions of iterations

In [ ]:
def create_dir(path):
  directory = os.path.dirname(path)
  try:
    os.stat(directory)
  except:
    os.mkdir(directory)

def main():
  for d in [15, 30, 50, 100]:
    print (f"starting FSS ({d})")
    search_space_initializer = UniformSSInitializer()
    file_path = os.path.dirname(os.path.abspath('Algoritms')) + os.sep + "Executions" + os.sep + f"{d}d" + os.sep
    result_path = os.path.dirname(os.path.abspath('Algoritms')) + os.sep + "Results" + os.sep + f"{d}d" + os.sep
    num_exec = 30
    school_size = 30
    num_iterations = 1000
    step_individual_init = 0.1
    step_individual_final = 0.0001
    step_volitive_init = 0.01
    step_volitive_final = 0.001
    min_w = 1
    w_scale = num_iterations / 2.0

    unimodal_funcs = [SphereFunction, RotatedHyperEllipsoidFunction, RosenbrockFunction, DixonPriceFunction, QuarticNoiseFunction]
    multimodal_funcs = [GeneralizedShwefelFunction, RastriginFunction, AckleyFunction, GriewankFunction, LeviFunction]
    regular_functions = unimodal_funcs + multimodal_funcs

    # regular_functions = [RosenbrockFunction, RastriginFunction]

    # Notice that for CEC Functions only the following dimensions are available:
    # 2, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100
    cec_functions = []

    create_dir(result_path)
    f_handle_csv = open(result_path + "/FSS_exec.csv", 'w+')
    writer_csv = csv.writer(f_handle_csv, delimiter=",")
    header = ['opt', 'func', 'exec_time'] + [f"run{str(i+1)}" for i in range(num_exec)]
    writer_csv.writerow(header)

    for benchmark_func in regular_functions:
      func = benchmark_func(d)
      if func.function_name != 'Perm':
        start = time.time()
        runs = run_experiments(num_iterations, school_size, num_exec, func,
                              search_space_initializer, step_individual_init,
                              step_individual_final, step_volitive_init,
                              step_volitive_final, min_w, w_scale, file_path)
        end = time.time()
      else:
        start = time.time()
        end = time.time()
        runs = [328.95400574534574, 326.04127506500237, 2.796904099932199e+98, 6.519735731856251e+34, 1073.1750127886419, 322.83118942310347, 554.163218331449, 331.2276397338817, 5.752455177962465e+100, 658.4360030610452,
                9.357742875932457e+51, 326.63211323297537, 1202.3334753853694, 104.36412088716655, 2.2123015131844664e+158, 761.852781822078, 329.85582612805916, 317.4486674714947, 545.2494634373231, 697.9959171293847,
                1013.57721548431, 4.636337105901376e+159, 320.1245071249878, 1.2763687989433055e+140, 329.28991627617864, 1225.3240699009973, 306.1989144851666, 356.2662413691464, 784.9468980768678, 2.4901661214111027e+197]
      row_csv = ['FSS', func.function_name, (end - start)] + [r for r in runs]
      writer_csv.writerow(row_csv)
      
    f_handle_csv.close()


def run_experiments(n_iter, school_size, num_runs, objective_function,
                    search_space_initializer, step_individual_init,
                    step_individual_final, step_volitive_init,
                    step_volitive_final, min_w, w_scale, save_dir):
  alg_name = "FSS"
  console_out = "Algorithm: {} Function: {} Execution: {} Best Cost: {}"
  if save_dir:
    create_dir(save_dir)
    f_handle_cost_iter = open(save_dir + "/FSS_" + objective_function.function_name + "_cost_iter.txt", 'w+')
    f_handle_cost_eval = open(save_dir + "/FSS_" + objective_function.function_name + "_cost_eval.txt", 'w+')
    runs = []

  for run in range(num_runs):
    opt1 = FSS(objective_function=objective_function, space_initializer=search_space_initializer,
                n_iter=n_iter, school_size=school_size, step_ind_init=step_individual_init,
                step_ind_final=step_individual_final, step_vol_init=step_volitive_init,
                step_vol_final=step_volitive_final, min_weight=min_w, weight_scale=w_scale)
    
    opt1.optimize()
    runs.append(opt1.best_fish.cost)
    print(console_out.format(alg_name, objective_function.function_name, run+1, opt1.best_fish.cost))

    temp_optimum_cost_tracking_iter = np.asmatrix(opt1.optimum_cost_tracking_iter)
    temp_optimum_cost_tracking_eval = np.asmatrix(opt1.optimum_cost_tracking_eval)

    if save_dir:
      np.savetxt(f_handle_cost_iter, temp_optimum_cost_tracking_iter, fmt='%.4e')
      np.savetxt(f_handle_cost_eval, temp_optimum_cost_tracking_eval, fmt='%.4e')

  if save_dir:
    f_handle_cost_iter.close()
    f_handle_cost_eval.close()

  return runs

main()

starting FSS (15)
Algorithm: FSS Function: Sphere Execution: 1 Best Cost: 5.852962741406872e-07
Algorithm: FSS Function: Sphere Execution: 2 Best Cost: 8.369380447579445e-08
Algorithm: FSS Function: Sphere Execution: 3 Best Cost: 4.353476550329123e-09
Algorithm: FSS Function: Sphere Execution: 4 Best Cost: 1.0056933083191062e-10
Algorithm: FSS Function: Sphere Execution: 5 Best Cost: 2.2875418916891103e-07
Algorithm: FSS Function: Sphere Execution: 6 Best Cost: 7.636815016308575e-09
Algorithm: FSS Function: Sphere Execution: 7 Best Cost: 4.5072812757046265e-09
Algorithm: FSS Function: Sphere Execution: 8 Best Cost: 7.801423555170602e-09
Algorithm: FSS Function: Sphere Execution: 9 Best Cost: 3.1181717296114744e-08
Algorithm: FSS Function: Sphere Execution: 10 Best Cost: 2.1994864205509802e-07
Algorithm: FSS Function: Sphere Execution: 11 Best Cost: 1.1271547127325112e-08
Algorithm: FSS Function: Sphere Execution: 12 Best Cost: 8.716161723490856e-10
Algorithm: FSS Function: Sphere Execu